## Goal
- ~Set up pandas dataframe of RV + MJD data to use in PCA~ - done!

## Steps
- ~Sort plates by n visits per target~ - done!
- ~Pick plate with n > 20 visits/target for initial pca~ - max 16

## Questions
- Should I remove targets which don't have the same number of visits? --> Yes

In [2]:
import numpy as np
import pandas as pd
import statistics as stat
from astropy.io import fits
from collections import Counter

In [3]:
# Read in FITS file

visit_hdus = fits.open('allVisit-r12-l33.fits')

# print(visit_hdus.info())

In [4]:
# HDU stuff

visit_header = visit_hdus[1].header
visit_data = visit_hdus[1].data

visit_hdus.close()

# print(visit_header)

In [5]:
# Set up variables for FITS file data

all_targets = list(visit_data['TARGET_ID'])
all_mjd = list(visit_data['MJD'])
all_obsvhelio = list(visit_data['OBSVHELIO']) # Heliocentric relative RV from 'observed spectrum template matching'
all_obsvrelerr = list(visit_data['OBSVRELERR']) # RV error of OBSVREL
all_plates = list(visit_data['PLATE'])

In [6]:
# Strip whitespace from Plate IDs

for i, s in enumerate(all_plates):
    all_plates[i] = s.strip()
    
# print(all_plates[:7])

In [7]:
# Create pandas dataframe for all data

all_data = [all_targets, all_plates, all_mjd, all_obsvhelio, all_obsvrelerr]

df = pd.DataFrame(all_data).transpose()
df.columns = ['Target ID', 'Plate ID', 'MJD', 'OBSVHELIO (km/s)', 'OBSVREL Error (km/s)']

In [8]:
# Show all relevant FITS file data

df.sort_values('OBSVHELIO (km/s)')

,Target ID,Plate ID,MJD,OBSVHELIO (km/s),OBSVREL Error (km/s)
1106307,apo25m.5118.URMINOR.2M15062489+6715577,8628,58298,-1768.998169,0.355434
467533,lco25m.5214.CARINA.2M06472679-5057077,10209,58212,-1640.690063,0.105672
786088,apo25m.4503.160+60.2M11002078+4809563,7348,56729,-1628.462891,0.217651
1026410,apo25m.5748.BOOTES1.2M13544089+1520421,10894,58292,-1616.208618,0.18248
1108939,apo25m.5118.URMINOR.2M15075464+6710445,8628,57468,-1580.105103,0.262306
...,...,...,...,...,...
47659,lco25m.5499.SMC3.2M00375577-7343181,10083,58028,999999.0,999999.0
380012,apo25m.4568.191-04.2M05522126+1734585,6762,56565,999999.0,999999.0
380013,apo25m.4568.191-04.2M05522126+1734585,6762,56566,999999.0,999999.0
1696691,apo25m.5110.M15.2M21293076+1206327,8706,58296,999999.0,999999.0


In [9]:
# Drop bad RV values

df_cutoff = df.loc[(df['OBSVHELIO (km/s)'] < 3000) & (df['OBSVREL Error (km/s)'] != 0) & (df['OBSVHELIO (km/s)'] > -3000)]

In [10]:
df_cutoff.sort_values('OBSVHELIO (km/s)')

,Target ID,Plate ID,MJD,OBSVHELIO (km/s),OBSVREL Error (km/s)
1106307,apo25m.5118.URMINOR.2M15062489+6715577,8628,58298,-1768.998169,0.355434
467533,lco25m.5214.CARINA.2M06472679-5057077,10209,58212,-1640.690063,0.105672
786088,apo25m.4503.160+60.2M11002078+4809563,7348,56729,-1628.462891,0.217651
1026410,apo25m.5748.BOOTES1.2M13544089+1520421,10894,58292,-1616.208618,0.18248
1108939,apo25m.5118.URMINOR.2M15075464+6710445,8628,57468,-1580.105103,0.262306
...,...,...,...,...,...
725074,apo25m.5270.ORPHAN-3.2M10123304+2625481,9663,57822,1518.61145,0.111901
725078,apo25m.5270.ORPHAN-3.2M10123304+2625481,9681,57820,1519.69812,0.121078
725073,apo25m.5270.ORPHAN-3.2M10123304+2625481,9663,57819,1520.107666,0.132766
725085,apo25m.5270.ORPHAN-3.2M10123304+2625481,9691,57826,1631.672119,0.468834


In [11]:
# Get list of unique plates

unique_plates = []

for i in Counter(df_cutoff['Plate ID']):
    if i not in unique_plates:
        unique_plates.append(i)
        
print("Number of unique plates:", len(unique_plates))

Number of unique plates: 2383


In [12]:
# Get mode number of visits per target on each plate

nvisits_mode = []
# filtered_plates = []

for p in unique_plates:
    dff = df_cutoff[df_cutoff['Plate ID'] == p]
    count_dict = Counter(dff['Target ID'])
    mode_visits = stat.mode(count_dict.values())
    nvisits_mode.append(mode_visits)
#    if mode_visits > 19:
#        filtered_plates.append(p)

In [13]:
# Create pandas df of mode number of visits per target on each plate

visitcount_data = [unique_plates, nvisits_mode]
visitcount_df = pd.DataFrame(visitcount_data).transpose()
visitcount_df.columns = ['Plate ID', 'Mode Visit Count/Target']

In [14]:
# Show plates with more than 10 visits per target

visitcount_df[visitcount_df['Mode Visit Count/Target'] > 10] #['Mode Visit Count/Target'].sort_values(ascending=False)

,Plate ID,Mode Visit Count/Target
70,9518,11
154,9244,11
234,9290,16
634,8907,14
1269,5631,11
1372,8112,12
1981,9860,12


In [27]:
# Create pandas df for plate 9290

plate9290_df = df_cutoff[df_cutoff['Plate ID'] == '9290']
plate8907_df = df_cutoff[df_cutoff['Plate ID'] == '8907']
plate8112_df = df_cutoff[df_cutoff['Plate ID'] == '8112']
#plate9290_df = df_cutoff[df_cutoff['Plate ID'] == '9290']
#plate9290_df = df_cutoff[df_cutoff['Plate ID'] == '9290']
#plate9290_df
multiplate_df = pd.concat([plate9290_df, plate8907_df, plate8112_df], axis=0)
multiplate_df

,Target ID,Plate ID,MJD,OBSVHELIO (km/s),OBSVREL Error (km/s)
164071,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57706,-86.565704,0.010245
164072,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57732,-86.38343,0.008206
164073,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57734,-86.386871,0.009015
164074,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57735,-86.532837,0.010742
164075,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57760,-86.366638,0.012035
...,...,...,...,...,...
1139307,apo25m.5118.URMINOR.2M15260674+6734563,8112,57879,0.493465,0.061337
1139308,apo25m.5118.URMINOR.2M15260674+6734563,8112,57886,0.285309,0.083303
1139309,apo25m.5118.URMINOR.2M15260674+6734563,8112,57887,0.420483,0.06766
1139310,apo25m.5118.URMINOR.2M15260674+6734563,8112,57916,0.399855,0.06662


In [28]:
# Check visit count per target

target_nvisits = Counter(multiplate_df['Target ID'])

In [29]:
# Identify all targets which have < 16 visits

target_nvisits_data = [list(target_nvisits.keys()), list(target_nvisits.values())]

target_nvisits_df = pd.DataFrame(target_nvisits_data).transpose()
target_nvisits_df.columns = ['Target ID', 'Visit Count']

#targets2drop = target_nvisits_df[target_nvisits_df['Visit Count'] < 16]
#targets2drop

In [30]:
#targets2drop_list = list(targets2drop['Target ID'])

# len(targets2drop_list)
sum(list(target_nvisits_df['Visit Count']))

10669

In [18]:
# Drop all targets which were visited < 16 times

for target in targets2drop_list:
    plate9290_df_clean = plate9290_df[(plate9290_df['Target ID'] != target)]
    # print(target)
    # print(plate9290_df_clean)
    plate9290_df = plate9290_df_clean

plate9290_df_clean

,Target ID,Plate ID,MJD,OBSVHELIO (km/s),OBSVREL Error (km/s)
164071,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57706,-86.5657,0.010245
164072,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57732,-86.3834,0.00820577
164073,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57734,-86.3869,0.00901516
164074,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57735,-86.5328,0.0107417
164075,apo25m.5226.150-08-RV.2M03252400+4614203,9290,57760,-86.3666,0.0120355
...,...,...,...,...,...
192942,apo25m.5226.150-08-RV.2M03415658+4626067,9290,58068,-26.4901,0.0115105
192943,apo25m.5226.150-08-RV.2M03415658+4626067,9290,58085,-26.5953,0.0122043
192944,apo25m.5226.150-08-RV.2M03415658+4626067,9290,58087,-26.604,0.0110477
192945,apo25m.5226.150-08-RV.2M03415658+4626067,9290,58114,-26.5992,0.0211242


In [19]:
# Number of individual stars on this plate

len(Counter(plate9290_df_clean['Target ID']))

213

In [31]:
# Convert df to csv file
multiplate_df.to_csv('multiplate_RVs.csv')
#plate9290_df_clean.to_csv('plate9290_RVs.csv') # Note that 0th column is preserved and must be removed when importing

In [21]:
# No longer relevant - setup for pandas df of visit count per target

# target_count = Counter(df['Target ID'])
# target_count = Counter(all_targets)

# filtered_targets = []
# filtered_targets_count = []

# for tc in target_count:
#     if target_count[tc] > 19:
#         filtered_targets.append(tc)
#         filtered_targets_count.append(target_count[tc])
        
# filtered_targets_data = [filtered_targets, filtered_targets_count]
# targets_df = pd.DataFrame(filtered_targets_data).transpose()
# targets_df.columns = ['Target ID', 'Visit Count']

# targets_df

# df[df['Target ID'] == 'apo25m.4230.M15.2M21342357+1215247']